In [7]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [8]:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test_QkPvNLx.csv')

In [9]:
all_df = train_df.append(test_df)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [70]:
get_end_limit = lambda x : x - 60

def add_median(x, day_index_df, col_name, period, tech = 0):
    el = get_end_limit(x)
    day_index_df = day_index_df[:el]
    if x < period[1]*7:
        return np.nan
    else:
        temp = day_index_df[:x]
    if tech == 0:
        return np.median(temp)
    else:
        if 7*period[0] == 0:
            return np.median(temp[-7*period[1]:])
        return np.median(temp[-7*period[1]:-7*period[0]])

def add_mean(x, day_index_df, col_name, period, tech = 0):
    el = get_end_limit(x)
    day_index_df = day_index_df[:el]
    if x < period[1]*7:
        return np.nan
    else:
        temp = day_index_df[:x]
    if tech == 0:
        return np.mean(temp)
    else:
        if 7*period[0] == 0:
            return np.mean(temp[-7*period[1]:])
        return np.mean(temp[-7*period[1]:-7*period[0]])

def make_feats(temp, col = 'Sales'):
    temp2 = temp.copy()
    day_toend_day_map = {x : get_end_limit(x) for x in temp2.Day_No}
    temp2.set_index('Day_No', drop = False, inplace = True)

    periods = [(0,1), (0,2), (0,3), (0,5), (0,8), (0,13), (0,21)]
    temp[f'{col}_median_60_lag'] = temp['Day_No'].apply(lambda x : add_median(x,
                                                                          temp2[col].values, col, (0,1/7), tech = 0))
    temp[f'{col}_mean_60_lag'] = temp['Day_No'].apply(lambda x : add_mean(x,
                                                                          temp2[col].values, col, (0,1/7), tech = 0))
    for p in periods:
       
        c_mean = f'{col}_mean_{p[0]}_{p[1]}'
        c_median = f'{col}_median_{p[0]}_{p[1]}'
       
        temp[c_mean] = temp['Day_No'].apply(lambda x : add_mean(x, temp2[col].values, col, p, tech =1))
        temp[c_median] = temp['Day_No'].apply(lambda x : add_median(x,
                                                       temp2[col].values, col, p, tech =1))
    return temp


In [77]:
made_feats = []
for i, v in all_df.groupby('Course_ID'):
    made_feats.append(make_feats(v))

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [78]:
made_feats_df = pd.concat(made_feats)

In [82]:
made_feats_df[['Sales_median_60_lag',
       'Sales_mean_60_lag', 'Sales_mean_0_1', 'Sales_median_0_1',
       'Sales_mean_0_2', 'Sales_median_0_2', 'Sales_mean_0_3',
       'Sales_median_0_3', 'Sales_mean_0_5', 'Sales_median_0_5',
       'Sales_mean_0_8', 'Sales_median_0_8', 'Sales_mean_0_13',
       'Sales_median_0_13', 'Sales_mean_0_21', 'Sales_median_0_21', 'ID']].to_csv('fib_med_feats.csv', index = False)